In [27]:
import pandas as pd
import matplotlib.pyplot as plt

In [28]:
bond  = pd.read_csv("../data/raw/Isin.csv")

In [29]:
bond.columns

Index(['IsinIdx', 'TickerIdx', 'ActualMaturityDateKey', 'IssueDateKey',
       'Seniority', 'Currency', 'ActivityGroup', 'Region', 'Activity',
       'RiskCaptain', 'Owner', 'CompositeRating', 'IndustrySector',
       'IndustrySubgroup', 'MarketIssue', 'IssuedAmount', 'CouponType'],
      dtype='object')

In [30]:
bond.head(2)

,IsinIdx,TickerIdx,ActualMaturityDateKey,IssueDateKey,Seniority,Currency,ActivityGroup,Region,Activity,RiskCaptain,Owner,CompositeRating,IndustrySector,IndustrySubgroup,MarketIssue,IssuedAmount,CouponType
0,0,238,20381231,20051129,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NR,Government,Sovereign,Domestic,1.246002e+09,STEP CPN
1,1,238,20331231,20051129,GOV,USD,FLOW LOCAL MARKET,AMERICAS,ARGENTINA,ARGENTINA,EMK ARGENTINA,NR,Government,Sovereign,Domestic,4.901086e+09,FIXED


In [31]:
bond.groupby("MarketIssue").size()

MarketIssue
Canadian              2
Domestic            788
Domestic mtn        640
Euro mtn           7374
Euro non-dollar    1654
Euro-dollar        3553
Euro-zone          2001
Global             5690
Priv placement     3146
UK gilt stock        43
US domestic        2216
US govt             169
US non-dollar         4
Yankee              114
dtype: int64

In [32]:
bond.loc[~bond.Seniority.isin(["GOV","SEC","SEN","SUB"]), "Seniority"] = "OTHER"
bond.loc[~bond.Activity.isin(["Asia","RETAIL","GBP SAS", "ARGENTINIA"]), "Activity"] = "OTHER"
bond.loc[bond.Currency.isin(
    (bond.groupby("Currency").size())[bond.groupby("Currency").size()<350].index.tolist()
), "Currency"] = "OTHER"

bond.loc[bond.CompositeRating.str.contains("C|D"), "CompositeRating"] = "LOWER"
# Risk captain
bond.loc[bond.RiskCaptain.isin(
    (bond.groupby("RiskCaptain").size())[bond.groupby("RiskCaptain").size()<200].index.tolist()
), "RiskCaptain"] = "OTHER"

In [33]:
data = pd.read_csv("../data/prepared/data_basic_1.csv", compression = "gzip")

C:\Users\kozodoin4.hub.RDC\AppData\Local\Continuum\anaconda3\envs\python3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [34]:
tmp = data.loc[data.CustomerInterest == 1]

In [35]:
tmp = tmp.loc[tmp.Week<61]

In [36]:
tmp.columns

Index(['PredictionIdx', 'CustomerIdx', 'IsinIdx', 'BuySell',
       'CustomerInterest', 'Week'],
      dtype='object')

In [37]:
full = tmp.merge(bond, on=["IsinIdx"], how="left")

In [38]:
final_table = pd.DataFrame({"CustomerIdx":full.CustomerIdx.unique()})

In [39]:
bondfeature_list = ["Seniority","Currency","ActivityGroup","Region","Activity", 
                    "RiskCaptain", "CompositeRating", "IndustrySector",
                   "MarketIssue"]

In [40]:
full.head()

,PredictionIdx,CustomerIdx,IsinIdx,BuySell,CustomerInterest,Week,TickerIdx,ActualMaturityDateKey,IssueDateKey,Seniority,...,Region,Activity,RiskCaptain,Owner,CompositeRating,IndustrySector,IndustrySubgroup,MarketIssue,IssuedAmount,CouponType
0,NaN,0,20909,Sell,1.0,45,1333,20200130,20130130,SEN,...,ASIA-TOKYO,OTHER,ASIA HIGH YIELD,ASIA HIGH YIELD,NR,Industrial,Diversified Manufact Op,Euro-dollar,4.000000e+08,FIXED
1,NaN,0,20910,Buy,1.0,52,2703,20200124,20130124,SEN,...,ASIA-TOKYO,OTHER,ASIA HIGH YIELD,ASIA HIGH YIELD,B+,Financial,Real Estate Oper/Develop,Euro-dollar,6.000000e+08,FIXED
2,NaN,0,21852,Buy,1.0,52,1512,20190110,20140110,SEN,...,ASIA-TOKYO,OTHER,ASIA HIGH YIELD,ASIA HIGH YIELD,B+,Financial,Real Estate Oper/Develop,Euro-dollar,1.000000e+09,FIXED
3,NaN,0,22294,Sell,1.0,60,2893,20180519,20140519,SEN,...,ASIA-TOKYO,OTHER,ASIA HIGH YIELD,ASIA HIGH YIELD,NR,Financial,Real Estate Oper/Develop,Euro-dollar,6.370270e+08,FIXED
4,NaN,0,22331,Buy,1.0,53,2009,20190604,20140604,SEN,...,ASIA-TOKYO,OTHER,ASIA HIGH YIELD,ASIA HIGH YIELD,B+,Financial,Real Estate Oper/Develop,Euro-dollar,3.000000e+08,FIXED


In [41]:
import scipy.stats

In [45]:
fac_data = full.groupby("CustomerIdx").agg(lambda x: scipy.stats.mode(x)[0][0])

C:\Users\kozodoin4.hub.RDC\AppData\Local\Continuum\anaconda3\envs\python3\lib\site-packages\scipy\stats\stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [50]:
fac_data.columns = fac_data.columns + "_mode"
fac_data = fac_data.reset_index()

In [52]:
fac_data.head()

,CustomerIdx,PredictionIdx_mode,IsinIdx_mode,BuySell_mode,CustomerInterest_mode,Week_mode,TickerIdx_mode,ActualMaturityDateKey_mode,IssueDateKey_mode,Seniority_mode,Currency_mode,ActivityGroup_mode,Region_mode,Activity_mode,RiskCaptain_mode,Owner_mode,CompositeRating_mode,IssuedAmount_mode,CouponType_mode
0,0,0,20909,Sell,1.0,45,1056,20180519,20130124,SEN,USD,FLOW LOCAL MARKET,ASIA-TOKYO,OTHER,ASIA HIGH YIELD,ASIA HIGH YIELD,NR,3.000000e+08,FIXED
1,1,0,12796,Sell,1.0,29,2480,20210523,20160523,SEN,USD,FLOW LOCAL MARKET,AMERICAS,OTHER,EMK BRAZIL CORP,EMK BRAZIL,BB-,6.750000e+09,FIXED
2,3,0,22536,Sell,1.0,55,325,20240619,20140619,SEN,EUR,FLOW G10,EUROPE,OTHER,FINANCIALS,DISTRESS,A-,1.250000e+09,FIXED
3,4,0,1914,Sell,1.0,25,31,20180820,20080820,SUB,EUR,FLOW G10,EUROPE,OTHER,FINANCIALS,INSCO CASH,BBB+,5.000000e+08,FIXED
4,5,0,18197,Buy,1.0,47,2317,20170418,20030214,SEN,EUR,FLOW G10,EUROPE,OTHER,FINANCIALS,SPANISH REGIONS,NR,5.000000e+08,FIXED


In [53]:
# list of features
excluded_features = ["PredictionIdx_mode", "CustomerInterest_mode", "Week_mode", "TickerIdx_mode", 
                     "ActualMaturityDateKey_mode", "IssueDateKey_mode", "IssuedAmount_mode"]
features = [var for var in fac_data.columns if var not in excluded_features]

In [54]:
fac_data = fac_data[features]

In [55]:
fac_data.head()

,CustomerIdx,IsinIdx_mode,BuySell_mode,Seniority_mode,Currency_mode,ActivityGroup_mode,Region_mode,Activity_mode,RiskCaptain_mode,Owner_mode,CompositeRating_mode,CouponType_mode
0,0,20909,Sell,SEN,USD,FLOW LOCAL MARKET,ASIA-TOKYO,OTHER,ASIA HIGH YIELD,ASIA HIGH YIELD,NR,FIXED
1,1,12796,Sell,SEN,USD,FLOW LOCAL MARKET,AMERICAS,OTHER,EMK BRAZIL CORP,EMK BRAZIL,BB-,FIXED
2,3,22536,Sell,SEN,EUR,FLOW G10,EUROPE,OTHER,FINANCIALS,DISTRESS,A-,FIXED
3,4,1914,Sell,SUB,EUR,FLOW G10,EUROPE,OTHER,FINANCIALS,INSCO CASH,BBB+,FIXED
4,5,18197,Buy,SEN,EUR,FLOW G10,EUROPE,OTHER,FINANCIALS,SPANISH REGIONS,NR,FIXED


In [56]:
fac_data.to_csv("../data/prepared/favorite_bonds_v2_60.csv", index=False, compression = "gzip")

In [14]:
for var in bondfeature_list:
    count_table = full.groupby(["CustomerIdx",var]).size().to_frame('count').reset_index()

    #count_table = count_table.merge(
    #    count_table.groupby("CustomerIdx").max().drop(var, axis=1).reset_index(), 
    #                  on =["CustomerIdx"], how="left")

    count_table.sort_values(["CustomerIdx", "count"], ascending=False).groupby("CustomerIdx").head(1)
    count_table = count_table.loc[:, ["CustomerIdx",var]]
    
    final_table = final_table.merge(count_table, on=["CustomerIdx"], how = "left")
    del(count_table)

In [15]:
final_table.head()

,CustomerIdx,Seniority,Currency,ActivityGroup,Region,Activity,RiskCaptain,CompositeRating,IndustrySector,MarketIssue
0,0,SEN,USD,FLOW G10,ASIA-TOKYO,OTHER,ASIA HIGH YIELD,B+,Financial,Euro-dollar
1,0,SEN,USD,FLOW G10,ASIA-TOKYO,OTHER,ASIA HIGH YIELD,B+,Industrial,Euro-dollar
2,0,SEN,USD,FLOW G10,ASIA-TOKYO,OTHER,ASIA HIGH YIELD,BBB,Financial,Euro-dollar
3,0,SEN,USD,FLOW G10,ASIA-TOKYO,OTHER,ASIA HIGH YIELD,BBB,Industrial,Euro-dollar
4,0,SEN,USD,FLOW G10,ASIA-TOKYO,OTHER,ASIA HIGH YIELD,BBB+,Financial,Euro-dollar


In [17]:
final_table.shape

(4823369432, 10)

In [25]:
del final_table

In [ ]:
final_table = final_table.drop_duplicates()

In [ ]:
final_table.to_csv("../data/prepared/favorite_bonds_v2_60.csv", index=False, compression = "gzip")